## Import Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
% pip install wandb

     |████████████████████████████████| 1.8MB 6.1MB/s 
     |████████████████████████████████| 81kB 14.1MB/s 
     |████████████████████████████████| 133kB 72.4MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
     |████████████████████████████████| 163kB 58.1MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=ab1a1fb5e33028cf67d129fa65143e47fa6c1f7ee57cd8fe663726db5c48d7ed
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built subprocess32


In [ ]:
from os.path import isfile, join
import numpy as np
import wandb
from utils import HGN_organize_data, HGN_divide_train_test
from code_HGN_text2pickup_train import train

In [ ]:
wandb.login()

wandb: Currently logged in as: awesomericky (use `wandb login --relogin` to force relogin)


True

## Load Preprocessed Data

In [ ]:
relative_data_directory_path = '/content/drive/MyDrive/Speech2Pickup/dataset1/data_text2pickup/preprocessed4HGN_text2pickup.npz'

npzfile = np.load(relative_data_directory_path)

img_idx = npzfile['arr_0']
sen_len = npzfile['arr_1']
text_inputs = npzfile['arr_2']
pos_outputs = npzfile['arr_3']

print('End loading MetaData')

End loading MetaData


## Load data and divide training and test set

In [ ]:
num_data = img_idx.shape[0]
dim_sentence = text_inputs.shape[1]
max_step_sentence = text_inputs.shape[2]

img_path = '/content/drive/MyDrive/Speech2Pickup/dataset1/train_img'
script_path = '/content/drive/MyDrive/Speech2Pickup/dataset1/train_heatmap'

total_images, total_heatmaps = HGN_organize_data(img_path, script_path, model_type='text2pickup');

args = dict()
args['img_idx'] = img_idx
args['sen_len'] = sen_len
args['text_inputs'] = text_inputs
args['pos_outputs'] = pos_outputs
args['num_data'] = num_data

idx_train, idx_test, num_train, num_test, \
train_img_idx, train_sen_len, train_text_inputs, train_pos_outputs, \
test_img_idx, test_sen_len, test_text_inputs, test_pos_outputs = \
HGN_divide_train_test(args=args, model_type='text2pickup')

Loading image..
Loading 1/478
Loading 2/478
Loading 3/478
Loading 4/478
Loading 5/478
Loading 6/478
Loading 7/478
Loading 8/478
Loading 9/478
Loading 10/478
Loading 11/478
Loading 12/478
Loading 13/478
Loading 14/478
Loading 15/478
Loading 16/478
Loading 17/478
Loading 18/478
Loading 19/478
Loading 20/478
Loading 21/478
Loading 22/478
Loading 23/478
Loading 24/478
Loading 25/478
Loading 26/478
Loading 27/478
Loading 28/478
Loading 29/478
Loading 30/478
Loading 31/478
Loading 32/478
Loading 33/478
Loading 34/478
Loading 35/478
Loading 36/478
Loading 37/478
Loading 38/478
Loading 39/478
Loading 40/478
Loading 41/478
Loading 42/478
Loading 43/478
Loading 44/478
Loading 45/478
Loading 46/478
Loading 47/478
Loading 48/478
Loading 49/478
Loading 50/478
Loading 51/478
Loading 52/478
Loading 53/478
Loading 54/478
Loading 55/478
Loading 56/478
Loading 57/478
Loading 58/478
Loading 59/478
Loading 60/478
Loading 61/478
Loading 62/478
Loading 63/478
Loading 64/478
Loading 65/478
Loading 66/478
Loa

## Start Training

In [10]:
restore_flag = 2
restore_epoch = 0
# raise ValueError('Check restore_flag and restore_epoch again!! If confident # this code')
"""
1) restore_flag == 0
- Restore encoder model weights only
- restore_epoch == 0

2) restore_flag == 1
- Restore total model weights
- restore_epoch != 0
"""
if restore_flag == 0 and restore_epoch != 0:
    raise ValueError('Check restore_flag and restore_epoch')
elif restore_flag == 1 and restore_epoch == 0:
    raise ValueError('Check restore_flag and restore_epoch')

wandb_config = {'batch_size': 8,
                'max_epoch': 170,
                'num_hg_Depth': 4,
                'dim_hg_feat': 256,
                'learning_rate': 0.00001,
                'dropout_rate': 0.0,
                'num_train': num_train,
                'num_test': num_test}

if restore_flag == 2:
    id = wandb.util.generate_id()
    print('Wandb id: {}'.format(id))
    with open('/content/drive/MyDrive/Speech2Pickup/dataset1_model/Text2Pickup_model_wo/wandbID.txt', 'w') as f:
        f.write(str(id))
elif restore_flag == 1:
    id = '#####'
# else:
#     raise ValueError('Unsupported restore_flag')

wandb_run = wandb.init(project='Speech2Pickup', name='Text2Pickup', id=id, resume="allow", config=wandb_config)

train(img_resize = 256, # must be fixed
      heatmap_resize = 64, # must be fixed
      dim_sentence = dim_sentence,
      max_step_sentence = max_step_sentence,
      batch_size = 8, 
      max_epoch = 170,
      num_train = num_train,
      save_stride = 1,
      num_hg_Depth = 4, # good be fixed
      dim_hg_feat = 256, # good be fixed
      dim_rnn_cell = 256, # good be fixed
      restore_flag = restore_flag,
      restore_path = '/content/drive/MyDrive/Speech2Pickup/dataset1_model/Text2Pickup_model_wo/model.ckpt',
      restore_epoch = restore_epoch,
      total_images = total_images,
      total_heatmaps = total_heatmaps,
      train_text_inputs = train_text_inputs,
      train_sen_len = train_sen_len,
      train_img_idx = train_img_idx,
      train_pos_outputs = train_pos_outputs,
      learning_rate = 0.00001, # good be fixed
      model_save_path = '/content/drive/MyDrive/Speech2Pickup/dataset1_model/Text2Pickup_model_wo/model.ckpt')

Wandb id: 26zr78bj




Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooli

KeyboardInterrupt: ignored

In [ ]:
# Reset tensor graph
# Finish wandb process
import tensorflow as tf

tf.reset_default_graph()
wandb.finish()

Batch train loss,5e-05
_step,3723
_runtime,26650
_timestamp,1607879637
Epoch train loss,5e-05


Batch train loss,█▆▄▄▃▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Epoch train loss,█▇▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
# Past result from the author

train(img_resize = 256, # must be fixed
      heatmap_resize = 64, # must be fixed
      dim_sentence = dim_sentence,
      max_step_sentence = max_step_sentence,
      batch_size = 8, 
      max_epoch = 300,
      num_train = num_train,
      save_stride = 10,
      num_hg_Depth = 4, # good be fixed
      dim_hg_feat = 256, # good be fixed
      dim_rnn_cell = 256, # good be fixed
      restore_flag = 0,
      restore_path = '',
      restore_epoch = 0,
      total_images = total_images,
      total_heatmaps = total_heatmaps,
      train_text_inputs = train_text_inputs,
      train_sen_len = train_sen_len,
      train_img_idx = train_img_idx,
      train_pos_outputs = train_pos_outputs,
      learning_rate = 0.00001) # good be fixed

Now ready to start the session.
batch loss : 0.0411909 -> about 5427.975 second left to finish this epoch
batch loss : 0.0185834 -> about 457.861 second left to finish this epoch
batch loss : 0.0202038 -> about 415.988 second left to finish this epoch
batch loss : 0.0180969 -> about 390.180 second left to finish this epoch
batch loss : 0.0202712 -> about 368.268 second left to finish this epoch
batch loss : 0.0195459 -> about 348.313 second left to finish this epoch
batch loss : 0.0165712 -> about 328.878 second left to finish this epoch
batch loss : 0.0190475 -> about 309.755 second left to finish this epoch
batch loss : 0.0198531 -> about 291.177 second left to finish this epoch
batch loss : 0.0185292 -> about 272.564 second left to finish this epoch
batch loss : 0.0194838 -> about 254.310 second left to finish this epoch
batch loss : 0.0169322 -> about 236.128 second left to finish this epoch
batch loss : 0.0169373 -> about 218.096 second left to finish this epoch
batch loss : 0.018

batch loss : 0.0060431 -> about 287.662 second left to finish this epoch
batch loss : 0.00339873 -> about 269.909 second left to finish this epoch
batch loss : 0.00431055 -> about 252.116 second left to finish this epoch
batch loss : 0.0141158 -> about 234.335 second left to finish this epoch
batch loss : 0.00485154 -> about 216.567 second left to finish this epoch
batch loss : 0.00604522 -> about 198.793 second left to finish this epoch
batch loss : 0.00579137 -> about 180.999 second left to finish this epoch
batch loss : 0.00433577 -> about 163.219 second left to finish this epoch
batch loss : 0.00534647 -> about 145.488 second left to finish this epoch
batch loss : 0.00464767 -> about 127.728 second left to finish this epoch
batch loss : 0.00981156 -> about 109.985 second left to finish this epoch
batch loss : 0.0102914 -> about 92.219 second left to finish this epoch
batch loss : 0.00917304 -> about 74.461 second left to finish this epoch
batch loss : 0.0059264 -> about 56.678 seco

batch loss : 0.0042252 -> about 146.247 second left to finish this epoch
batch loss : 0.00840403 -> about 128.414 second left to finish this epoch
batch loss : 0.00652264 -> about 110.569 second left to finish this epoch
batch loss : 0.0146243 -> about 92.713 second left to finish this epoch
batch loss : 0.00636694 -> about 74.855 second left to finish this epoch
batch loss : 0.0104167 -> about 56.990 second left to finish this epoch
batch loss : 0.00180481 -> about 39.123 second left to finish this epoch
batch loss : 0.00365818 -> about 21.261 second left to finish this epoch
batch loss : 0.00332098 -> about 3.394 second left to finish this epoch
('current epoch : 9', ', current train loss : 0.00625545213552')
batch loss : 0.00513208 -> about 468.817 second left to finish this epoch
batch loss : 0.00850351 -> about 415.563 second left to finish this epoch
batch loss : 0.00265272 -> about 398.069 second left to finish this epoch
batch loss : 0.00641784 -> about 380.257 second left to f

batch loss : 0.00306799 -> about 3.059 second left to finish this epoch
('current epoch : 13', ', current train loss : 0.00365014836958')
batch loss : 0.00153681 -> about 532.361 second left to finish this epoch
batch loss : 0.00254444 -> about 510.746 second left to finish this epoch
batch loss : 0.00323756 -> about 489.155 second left to finish this epoch
batch loss : 0.00167175 -> about 466.711 second left to finish this epoch
batch loss : 0.0058559 -> about 444.880 second left to finish this epoch
batch loss : 0.00436605 -> about 423.108 second left to finish this epoch
batch loss : 0.00356909 -> about 401.310 second left to finish this epoch
batch loss : 0.00144523 -> about 379.233 second left to finish this epoch
batch loss : 0.00227549 -> about 357.070 second left to finish this epoch
batch loss : 0.00879056 -> about 335.050 second left to finish this epoch
batch loss : 0.00341763 -> about 313.081 second left to finish this epoch
batch loss : 0.00351238 -> about 291.046 second l

batch loss : 0.000845772 -> about 395.764 second left to finish this epoch
batch loss : 0.00471846 -> about 374.341 second left to finish this epoch
batch loss : 0.00267353 -> about 353.181 second left to finish this epoch
batch loss : 0.000774498 -> about 331.635 second left to finish this epoch
batch loss : 0.00222605 -> about 310.039 second left to finish this epoch
batch loss : 0.00455818 -> about 288.403 second left to finish this epoch
batch loss : 0.000527716 -> about 266.636 second left to finish this epoch
batch loss : 0.0007826 -> about 242.938 second left to finish this epoch
batch loss : 0.000941448 -> about 216.251 second left to finish this epoch
batch loss : 0.00071534 -> about 191.326 second left to finish this epoch
batch loss : 0.00357318 -> about 167.479 second left to finish this epoch
batch loss : 0.00242748 -> about 147.672 second left to finish this epoch
batch loss : 0.000542222 -> about 127.617 second left to finish this epoch
batch loss : 0.00143081 -> about 1

...

batch loss : 2.46027e-05 -> about 17.631 second left to finish this epoch
batch loss : 3.74629e-05 -> about 2.815 second left to finish this epoch
('current epoch : 145', ', current train loss : 5.68217978903e-05')
batch loss : 3.78775e-05 -> about 361.943 second left to finish this epoch
batch loss : 3.23514e-05 -> about 343.578 second left to finish this epoch
batch loss : 2.95365e-05 -> about 328.850 second left to finish this epoch
batch loss : 2.25258e-05 -> about 314.106 second left to finish this epoch
batch loss : 3.05632e-05 -> about 299.083 second left to finish this epoch
batch loss : 2.21192e-05 -> about 284.283 second left to finish this epoch
batch loss : 2.36475e-05 -> about 269.407 second left to finish this epoch
batch loss : 2.60451e-05 -> about 254.584 second left to finish this epoch
batch loss : 3.06136e-05 -> about 239.790 second left to finish this epoch
batch loss : 3.61305e-05 -> about 224.938 second left to finish this epoch
batch loss : 4.38522e-05 -> about 2

batch loss : 2.18922e-05 -> about 298.941 second left to finish this epoch
batch loss : 2.27803e-05 -> about 284.075 second left to finish this epoch
batch loss : 2.15718e-05 -> about 269.319 second left to finish this epoch
batch loss : 2.5339e-05 -> about 254.498 second left to finish this epoch
batch loss : 2.86893e-05 -> about 239.656 second left to finish this epoch
batch loss : 3.25537e-05 -> about 224.835 second left to finish this epoch
batch loss : 4.8805e-05 -> about 210.054 second left to finish this epoch
batch loss : 2.29665e-05 -> about 195.265 second left to finish this epoch
batch loss : 2.2166e-05 -> about 180.480 second left to finish this epoch
batch loss : 3.94648e-05 -> about 165.693 second left to finish this epoch
batch loss : 4.36609e-05 -> about 150.897 second left to finish this epoch
batch loss : 1.98164e-05 -> about 136.100 second left to finish this epoch
batch loss : 2.69195e-05 -> about 121.291 second left to finish this epoch
batch loss : 4.10729e-05 -> 

batch loss : 1.72116e-05 -> about 210.200 second left to finish this epoch
batch loss : 1.65372e-05 -> about 195.378 second left to finish this epoch
batch loss : 0.00013946 -> about 180.541 second left to finish this epoch
batch loss : 4.04796e-05 -> about 165.727 second left to finish this epoch
batch loss : 2.09603e-05 -> about 150.923 second left to finish this epoch
batch loss : 1.7969e-05 -> about 136.124 second left to finish this epoch
batch loss : 9.95178e-05 -> about 121.320 second left to finish this epoch
batch loss : 3.29073e-05 -> about 106.509 second left to finish this epoch
batch loss : 4.18255e-05 -> about 91.692 second left to finish this epoch
batch loss : 2.63741e-05 -> about 76.876 second left to finish this epoch
batch loss : 2.56479e-05 -> about 62.063 second left to finish this epoch
batch loss : 3.7586e-05 -> about 47.251 second left to finish this epoch
batch loss : 8.01115e-05 -> about 32.438 second left to finish this epoch
batch loss : 5.78649e-05 -> about

batch loss : 4.60938e-05 -> about 106.474 second left to finish this epoch
batch loss : 0.000116401 -> about 91.668 second left to finish this epoch
batch loss : 3.13063e-05 -> about 76.863 second left to finish this epoch
batch loss : 4.2867e-05 -> about 62.055 second left to finish this epoch
batch loss : 4.04995e-05 -> about 47.247 second left to finish this epoch
batch loss : 4.07299e-05 -> about 32.436 second left to finish this epoch
batch loss : 6.93641e-05 -> about 17.625 second left to finish this epoch
batch loss : 3.18264e-05 -> about 2.814 second left to finish this epoch
('current epoch : 158', ', current train loss : 5.628553116e-05')
batch loss : 0.000139049 -> about 356.033 second left to finish this epoch
batch loss : 6.37433e-05 -> about 343.149 second left to finish this epoch
batch loss : 2.9025e-05 -> about 328.677 second left to finish this epoch
batch loss : 8.45352e-05 -> about 313.981 second left to finish this epoch
batch loss : 3.34732e-05 -> about 299.196 se

batch loss : 2.69019e-05 -> about 17.659 second left to finish this epoch
batch loss : 3.41635e-05 -> about 2.820 second left to finish this epoch
('current epoch : 162', ', current train loss : 3.97461925034e-05')
batch loss : 2.04938e-05 -> about 358.314 second left to finish this epoch
batch loss : 5.24072e-05 -> about 344.332 second left to finish this epoch
batch loss : 6.49524e-05 -> about 329.272 second left to finish this epoch
batch loss : 2.41539e-05 -> about 314.433 second left to finish this epoch
batch loss : 2.30453e-05 -> about 299.551 second left to finish this epoch
batch loss : 7.62306e-05 -> about 284.755 second left to finish this epoch
batch loss : 0.000623336 -> about 269.887 second left to finish this epoch
batch loss : 3.43013e-05 -> about 255.090 second left to finish this epoch
batch loss : 3.86594e-05 -> about 240.287 second left to finish this epoch
batch loss : 3.91656e-05 -> about 225.436 second left to finish this epoch
batch loss : 3.11464e-05 -> about 2